In [2]:
# Need to restart after:
!pip install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.2/206.2 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Download file from Google Drive to colab directory
!pip install gdown
file_id = "1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0"
!gdown "https://drive.google.com/file/d/1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_topics10000_chronological.zip" --fuzzy

Downloading...
From (original): https://drive.google.com/uc?id=1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0
From (redirected): https://drive.google.com/uc?id=1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0&confirm=t&uuid=178229c8-aeb6-45c5-8810-6afadafc769c
To: /content/temporal_belief_analysis/pd_corpus_with_topics10000_chronological.zip
100% 841M/841M [00:07<00:00, 112MB/s]


In [2]:
# Unzip with python:
import zipfile
zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_topics10000_chronological.zip").extractall("/content/temporal_belief_analysis")

In [5]:
# For runpod-jupyter or local (run twice)
import sys
import os

# Change to the correct working directory (workspace if runpod, content if colab)
os.chdir('/content/temporal_belief_analysis/notebooks')
print("Changed working directory to:", os.getcwd())

# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Comment out if in colab:
from temporal_belief.core.timeline_building import TimelineBuilder

Changed working directory to: /content/temporal_belief_analysis/notebooks


In [4]:
# For colab:
from temporal_belief_analysis.src.temporal_belief.core.timeline_building import TimelineBuilder

In [6]:
# Run twice
# import unsloth
# import unsloth_zoo
from convokit import Corpus, download
import convokit

Error from Unsloth: NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.


/usr/local/lib/python3.11/dist-packages/convokit/utterance_simulator/unslothUtteranceSimulatorModel.py:2: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


In [7]:
# Load a corpus:
# corpus = Corpus(filename="/Users/leonidas/.convokit/saved-corpora/pd_corpus_with_stances1000_chronological")
corpus = Corpus(filename="/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological")

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


In [ ]:
!pip install scipy
!pip install statsmodels

In [8]:
def filter_for_change_detection(timelines, min_posts_per_topic=5, min_topics_per_user=2, min_confidence=0.0):
    """Filter timelines to only include users/topics suitable for change detection"""
    filtered_timelines = {}

    for user_id, user_timeline in timelines.items():
        filtered_user_timeline = {}

        for topic, topic_posts in user_timeline.items():
            # Filter by confidence (if you have access to corpus here)
            reliable_posts = {}
            for utt_id, stance in topic_posts.items():
                # You'd need to pass corpus or confidence scores here
                # For now, assume all posts are reliable
                reliable_posts[utt_id] = stance

            # Check minimum posts per topic
            if len(reliable_posts) >= min_posts_per_topic:
                filtered_user_timeline[topic] = reliable_posts

        # Check minimum topics per user
        if len(filtered_user_timeline) >= min_topics_per_user:
            filtered_timelines[user_id] = filtered_user_timeline

    return filtered_timelines

In [86]:
class ConversationWindowExtractor:
    def __init__(self, corpus, timelines):
        self.corpus = corpus
        self.timelines = timelines

    def get_user_conversations_chronological(self, corpus, speaker_id):
      """Get all conversations for a user in chronological order."""

      # Get all conversations where the speaker participated
      user_conversations = [convo for convo in corpus.iter_conversations()
                          if speaker_id in [utt.speaker.id for utt in convo.iter_utterances()]]

      # Sort conversations by their earliest timestamp
      user_conversations.sort(key=lambda convo: min(utt.timestamp for utt in convo.iter_utterances()))

      return user_conversations

    def get_conversation_around_change_point(self, corpus, change_points):
      prior_convos = []

      # Get utterance by ID, then get its speaker ID
      utterance = corpus.get_utterance(change_points[0][1])
      speaker_id = utterance.speaker.id

      # put conversations in chronological order in a list
      user_conversations = self.get_user_conversations_chronological(corpus, speaker_id)

      # get conversation id from utterance
      conversation = corpus.get_conversation(conversation_id)
      conversation_id = conversation.id

      # find the index of the convo, and return the convo id of the 3 prior convos
      for i, convo in enumerate(user_conversations):
        if conversation_id == user_conversations[i].id:
          prior_convos.append(user_conversations[i-2])
          prior_convos.append(user_conversations[i-1])

      prior_convos.append(conversation_id)

      return prior_convos

    def get_conversations_around_change(self, user_id, topic, change, window_size=3):
        """
        Get conversations before/after a belief change based on conversation count

        Args:
            user_id: The user who had the belief change
            topic: The topic where change occurred
            change: The significant change object from detect_changes_with_significance()
            window_size: Number of conversations before AND after change to include

        Returns:
            {
                'before_change': [conversation_data, ...],
                'after_change': [conversation_data, ...],
                'change_position': int,
                'change_utterance_id': str
            }
        """

        # Get ordered timeline for this user/topic
        topic_timeline = self.timelines[user_id][topic]
        timeline_items = list(topic_timeline.items())  # [(utterance_id, stance), ...]

        # Find position of the change utterance
        change_utterance_id = change['utterance_id']
        change_position = None

        for i, (utterance_id, stance) in enumerate(timeline_items):
            if utterance_id == change_utterance_id:
                change_position = i
                break

        if change_position is None:
            print(f"Warning: Change utterance {change_utterance_id} not found in timeline")
            return None

        print(f"Found change at position {change_position} of {len(timeline_items)} total utterances")

        # Extract conversations in window
        before_conversations = []
        after_conversations = []

        # Get conversations BEFORE change
        start_before = max(0, change_position - window_size)
        print(f"Looking for conversations before change: positions {start_before} to {change_position-1}")

        for i in range(start_before, change_position):
            utterance_id = timeline_items[i][0]
            conv_data = self._extract_conversation_from_utterance(utterance_id, user_id)
            if conv_data:
                before_conversations.append(conv_data)
                print(f"  Found conversation before: {conv_data['conversation_id']}")

        # Get conversations AFTER change
        end_after = min(len(timeline_items), change_position + window_size + 1)
        print(f"Looking for conversations after change: positions {change_position+1} to {end_after-1}")

        for i in range(change_position + 1, end_after):
            utterance_id = timeline_items[i][0]
            conv_data = self._extract_conversation_from_utterance(utterance_id, user_id)
            if conv_data:
                after_conversations.append(conv_data)
                print(f"  Found conversation after: {conv_data['conversation_id']}")

        return {
            'before_change': before_conversations,
            'after_change': after_conversations,
            'change_position': change_position,
            'change_utterance_id': change_utterance_id,
            'total_timeline_length': len(timeline_items),
            'window_size': window_size
        }

    def _extract_conversation_from_utterance(self, utterance_id, user_id):
        """Extract conversation data from a specific utterance"""
        try:
            utterance = self.corpus.get_utterance(utterance_id)
            if not utterance:
                return None

            conversation = utterance.get_conversation()
            if not conversation:
                return None

            op_post = conversation.get_root()
            if not op_post:
                return None

            user_replies = []

            # Get ALL user replies in this conversation (not just the target utterance)
            for utt in conversation.iter_utterances():
                if utt.speaker.id == user_id and utt.id != op_post.id:
                    user_replies.append(utt)

            if not user_replies:
                return None

            return {
                'conversation_id': conversation.id,
                'op_post': op_post,
                'user_replies': user_replies,
                'target_utterance_id': utterance_id,  # Which utterance led us to this conversation
                'topic': conversation.meta.get('detected_topic', 'unknown') if conversation.meta else 'unknown'
            }

        except Exception as e:
            print(f"Error extracting conversation for utterance {utterance_id}: {e}")
            return None

    def extract_rooted_path_units_from_window(self, window_data):
        """
        Extract rooted path units from conversations around a belief change.
        Uses your existing conversation extraction approach.
        """
        def process_conversations(conversations):
            processed = []

            for conv_data in conversations:
                # Use the conversation you ALREADY extracted in your working method
                conversation = conv_data['op_post'].get_conversation()  # Your existing approach

                if not conversation:
                    continue

                # Extract rooted path units for this conversation
                rooted_units = self._extract_rooted_units_from_conversation(conversation)

                # Get OP text
                op_text = conv_data['op_post'].text if conv_data['op_post'] else ""

                processed.append({
                    'conversation_id': conv_data['conversation_id'],
                    'rooted_units': rooted_units,
                    'op_text': op_text,
                    'topic': conv_data.get('topic', 'unknown'),
                    'original_conv_data': conv_data
                })

            return processed

        return {
            'before_change': process_conversations(window_data['before_change']),
            'after_change': process_conversations(window_data['after_change']),
            'change_info': {
                'change_position': window_data['change_position'],
                'change_utterance_id': window_data['change_utterance_id']
            }
        }

    def _extract_rooted_units_from_conversation(self, conversation):
        """
        Extract rooted path units from a single conversation.
        Every speaker gets their own rooted path unit.

        Returns:
            dict: {speaker_id: [list_of_utterances]}
        """
        rooted_units = {}

        for utterance in conversation.iter_utterances():
            speaker_id = utterance.speaker.id

            if speaker_id not in rooted_units:
                rooted_units[speaker_id] = []

            rooted_units[speaker_id].append(utterance)

        return rooted_units

    def preprocess_rooted_units_for_interplay(self, rooted_units_data):
        """
        Preprocess rooted path units for linguistic interplay calculation.

        Args:
            rooted_units_data: Output from extract_rooted_path_units_from_window()

        Returns:
            {
                'before_change': [
                    {
                        'conversation_id': str,
                        'interplay_pairs': [
                            {
                                'speaker_id': str,
                                'unit_text': str,  # Combined text of all speaker's utterances
                                'op_text': str,    # Original post text
                                'num_utterances': int
                            }, ...
                        ]
                    }, ...
                ],
                'after_change': [...] # same structure
            }
        """
        def process_rooted_units(conversations_with_units):
            processed = []

            for conv_data in conversations_with_units:
                interplay_pairs = []

                for speaker_id, utterances in conv_data['rooted_units'].items():
                    # Combine all utterances from this speaker
                    unit_text = self._combine_utterances_text(utterances)

                    interplay_pairs.append({
                        'speaker_id': speaker_id,
                        'unit_text': unit_text,
                        'op_text': conv_data['op_text'],
                        'num_utterances': len(utterances)
                    })

                processed.append({
                    'conversation_id': conv_data['conversation_id'],
                    'topic': conv_data['topic'],
                    'interplay_pairs': interplay_pairs
                })

            return processed

        return {
            'before_change': process_rooted_units(rooted_units_data['before_change']),
            'after_change': process_rooted_units(rooted_units_data['after_change']),
            'change_info': rooted_units_data['change_info']
        }

    def _combine_utterances_text(self, utterances):
        """
        Combine all utterances from a speaker into one text block.

        Args:
            utterances: List of ConvoKit utterances

        Returns:
            str: Combined text
        """
        return " ".join([utt.text for utt in utterances])


    def print_window_summary(self, window_data):
        """Helper method to print a summary of extracted conversations"""
        if not window_data:
            print("No window data to summarize")
            return

        print(f"\n=== Conversation Window Summary ===")
        print(f"Change occurred at position {window_data['change_position']} of {window_data['total_timeline_length']} utterances")
        print(f"Change utterance ID: {window_data['change_utterance_id']}")
        print(f"Window size: {window_data.get('window_size', 'unknown')}")

        print(f"\nConversations BEFORE change: {len(window_data['before_change'])}")
        for i, conv in enumerate(window_data['before_change']):
            print(f"  {i+1}. Conversation {conv['conversation_id']}: {len(conv['user_replies'])} user replies")

        print(f"\nConversations AFTER change: {len(window_data['after_change'])}")
        for i, conv in enumerate(window_data['after_change']):
            print(f"  {i+1}. Conversation {conv['conversation_id']}: {len(conv['user_replies'])} user replies")

In [61]:
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import fdrcorrection
from collections import Counter
import logging

class ChangeDetector:
    """Sliding window change detection with proper statistical significance."""

    def __init__(self, window_size=3, significance_level=0.05):
        self.window_size = window_size
        self.alpha = significance_level
        self.stance_values = {
            'strongly_against': -2, 'moderately_against': -1,
            'neutral': 0, 'moderately_favor': 1, 'strongly_favor': 2
        }

    def detect_simple_stance_changes(self, topic_timeline):

        if len(topic_timeline) < 2:
            return []

        changes = []
        timeline_items = list(topic_timeline.items())  # Convert to list of (utterance_id, stance) pairs

        for i in range(1, len(timeline_items)):
            current_utterance_id, current_stance = timeline_items[i]
            previous_utterance_id, previous_stance = timeline_items[i-1]

            # Check if stance changed
            if current_stance != previous_stance:
                change = {
                    'position': i,
                    'current_utterance_id': current_utterance_id,
                    'previous_utterance_id': previous_utterance_id,
                    'from_stance': previous_stance,
                    'to_stance': current_stance,
                    'change_type': self._classify_change_direction(previous_stance, current_stance),
                    'change_magnitude': self._calculate_simple_magnitude(previous_stance, current_stance)
                }
                changes.append(change)

        return changes

    def detect_persistent_changes(self, topic_timeline):
        """Detect persistent changes in stance."""

        # Convert to (utt_id, detected_stance) tuple
        topic_timeline_list = list(topic_timeline.items())

        # Collect the tuples where the stance is persistent across n utterances
        change_points = []

        for i in range(len(topic_timeline)):
          # if current stance is different than prior
          if topic_timeline[i][1] != topic_timeline[i-1][1]:
            # Check if change persists for more than 1 post
            if topic_timeline[i][1] == topic_timeline[i+1][1]:
              change_index = i
              utt_id = topic_timeline[i][0]
              change_point = (change_index, utt_id)
              change_points.append(change_point)
              print(f"Current:{topic_timeline[i][1]}, Previous: {topic_timeline[i-1][1]} and Next:{topic_timeline[i+1][1]}")
              # remove break
              break

        return change_points

    def _classify_change_direction(self, from_stance, to_stance):
        """Classify the direction of stance change."""
        from_value = self.stance_values.get(from_stance, 0)
        to_value = self.stance_values.get(to_stance, 0)

        if to_value > from_value:
            return 'more_favorable'
        elif to_value < from_value:
            return 'less_favorable'
        else:
            return 'neutral_shift'

    def _calculate_simple_magnitude(self, from_stance, to_stance):
        """Calculate the magnitude of stance change."""
        from_value = self.stance_values.get(from_stance, 0)
        to_value = self.stance_values.get(to_stance, 0)
        return abs(to_value - from_value)

    def detect_changes_with_significance(self, topic_timeline):
        """Detect changes with statistical significance testing."""

        if len(topic_timeline) < self.window_size * 2:
            return [], [], []

        # Convert to lists to maintain order and get IDs
        timeline_items = list(topic_timeline.items())  # [(utterance_id, stance), ...]
        stance_sequence = [self.stance_values.get(stance, 0) for _, stance in timeline_items]

        potential_changes = []
        p_values = []

        # Sliding window approach
        for i in range(self.window_size, len(stance_sequence) - self.window_size):

            # Left window (before potential change)
            left_window = stance_sequence[i - self.window_size:i]

            # Right window (after potential change)
            right_window = stance_sequence[i:i + self.window_size]

            # Statistical test: Are these two windows significantly different?
            statistic, p_value = self.two_sample_test(left_window, right_window)

            p_values.append(p_value)

            # Store potential change info with just the key utterance ID
            change_magnitude = abs(np.mean(right_window) - np.mean(left_window))
            potential_changes.append({
                'position': i,
                'utterance_id': timeline_items[i][0],  # The utterance where change detected
                'p_value': p_value,
                'test_statistic': statistic,
                'magnitude': change_magnitude,
                'left_mean': np.mean(left_window),
                'right_mean': np.mean(right_window),
                'left_window': left_window.copy(),
                'right_window': right_window.copy()
            })

        # Apply FDR correction to all p-values
        if not p_values:
            return [], [], []

        rejected, p_corrected = self.multiple_testing_correction(p_values)

        # Keep only changes that survive FDR correction
        significant_changes = []
        for i, change in enumerate(potential_changes):
            if rejected[i]:  # Survives FDR correction
                change.update({
                    'p_corrected': p_corrected[i],
                    'statistically_significant': True,
                    'survives_fdr_correction': True,
                    'significance_level': self.alpha
                })
                significant_changes.append(change)

        return significant_changes, p_values, p_corrected

    def two_sample_test(self, left_window, right_window):
        """Statistical test for difference between two windows."""
        # Use Mann-Whitney U test (non-parametric, more robust)
        try:
            statistic, p_value = mannwhitneyu(left_window, right_window,
                                            alternative='two-sided')
            return statistic, p_value
        except ValueError:
            # Fallback to t-test if Mann-Whitney fails
            statistic, p_value = ttest_ind(left_window, right_window)
            return statistic, p_value

    def multiple_testing_correction(self, p_values):
        """Correct for multiple testing using Benjamini-Hochberg."""
        rejected, p_corrected = fdrcorrection(p_values, alpha=self.alpha)
        return rejected, p_corrected

    # def analyze_user_belief_changes(self, user_timeline):
    #     """Analyze belief changes across all topics for a user."""
    #     all_changes = {}
    #
    #     for topic, topic_timeline in user_timeline.items():
    #         changes = self.detect_changes_with_significance(topic_timeline)
    #         all_changes[topic] = changes
    #
    #     return all_changes

    def analyze_user_belief_changes(self, user_timeline):
        """Analyze belief changes across all topics for a user.

        Args:
            user_timeline: Dict of {topic: {utterance_id: stance}}

        Returns:
            Dict with changes by topic and total count
        """
        all_changes = {}
        total_changes = 0

        for topic, topic_timeline in user_timeline.items():
            significant_changes, p_values, p_corrected = self.detect_changes_with_significance(topic_timeline)
            all_changes[topic] = significant_changes
            total_changes += len(significant_changes)

        return {
            'changes_by_topic': all_changes,
            'total_changes': total_changes
        }

    def analyze_all_users_belief_changes(self, timelines):
        """Analyze belief changes across all users.

        Args:
            timelines: Dict of {user_id: {topic: {utterance_id: stance}}}

        Returns:
            Dict with changes by user and total count
        """
        all_user_changes = {}
        total_changes = 0

        for user_id, user_timeline in timelines.items():
            user_result = self.analyze_user_belief_changes(user_timeline)
            all_user_changes[user_id] = user_result
            total_changes += user_result['total_changes']

        return {
            'changes_by_user': all_user_changes,
            'total_changes': total_changes
        }

In [12]:
# Mock corpus-like objects for testing
class MockUtterance:
    def __init__(self, id, speaker_id, text, timestamp, conversation):
        self.id = id
        self.speaker = MockSpeaker(speaker_id)
        self.text = text
        self.timestamp = timestamp
        self._conversation = conversation

    def get_conversation(self):
        return self._conversation

class MockSpeaker:
    def __init__(self, speaker_id):
        self.id = speaker_id

class MockConversation:
    def __init__(self, id, root_post, all_utterances):
        self.id = id
        self._root = root_post
        self._utterances = all_utterances
        self.meta = {'detected_topic': 'taxation and government spending'}

    def get_root(self):
        return self._root

    def iter_utterances(self):
        return iter(self._utterances)

class MockCorpus:
    def __init__(self):
        # Create mock conversations with realistic political discussion
        self.utterances = {}
        self.conversations = {}
        self._setup_mock_data()

    def get_utterance(self, utterance_id):
        return self.utterances.get(utterance_id)

    def _setup_mock_data(self):
        # Create 3 conversations for testing

        # Conversation 1: Tax policy discussion
        conv1_utterances = []
        op1 = MockUtterance(
            id="op_conv1",
            speaker_id="original_poster_1",
            text="I think we should raise taxes on the wealthy to fund infrastructure. The current system isn't working.",
            timestamp=None,
            conversation=None
        )

        user_reply1 = MockUtterance(
            id="user_reply_conv1",
            speaker_id="TestUser",
            text="I disagree with raising taxes. The wealthy already pay their fair share and higher taxes will hurt economic growth.",
            timestamp=None,
            conversation=None
        )

        conv1_utterances = [op1, user_reply1]
        conv1 = MockConversation("conv_1", op1, conv1_utterances)

        # Set conversation references
        for utt in conv1_utterances:
            utt._conversation = conv1

        # Conversation 2: Healthcare spending
        op2 = MockUtterance(
            id="op_conv2",
            speaker_id="original_poster_2",
            text="Government healthcare spending is out of control. We need to cut Medicare and focus on private solutions.",
            timestamp=None,
            conversation=None
        )

        user_reply2 = MockUtterance(
            id="user_reply_conv2",
            speaker_id="TestUser",
            text="Medicare is essential for seniors. We shouldn't cut it, but maybe we can find efficiencies without reducing benefits.",
            timestamp=None,
            conversation=None
        )

        conv2_utterances = [op2, user_reply2]
        conv2 = MockConversation("conv_2", op2, conv2_utterances)

        for utt in conv2_utterances:
            utt._conversation = conv2

        # Conversation 3: Budget discussion
        op3 = MockUtterance(
            id="op_conv3",
            speaker_id="original_poster_3",
            text="The federal budget deficit is unsustainable. We need major spending cuts across all departments.",
            timestamp=None,
            conversation=None
        )

        user_reply3 = MockUtterance(
            id="user_reply_conv3",
            speaker_id="TestUser",
            text="You're absolutely right. Government spending is completely out of control and we need dramatic cuts immediately.",
            timestamp=None,
            conversation=None
        )

        conv3_utterances = [op3, user_reply3]
        conv3 = MockConversation("conv_3", op3, conv3_utterances)

        for utt in conv3_utterances:
            utt._conversation = conv3

        # Store all utterances and conversations
        all_utterances = conv1_utterances + conv2_utterances + conv3_utterances
        for utt in all_utterances:
            self.utterances[utt.id] = utt

        self.conversations["conv_1"] = conv1
        self.conversations["conv_2"] = conv2
        self.conversations["conv_3"] = conv3

# Mock timeline data showing a clear stance change
mock_timelines = {
    "TestUser": {
        "taxation and government spending": {
            "user_reply_conv1": "moderately_against",  # Position 0: Against tax increases
            "user_reply_conv2": "neutral",             # Position 1: Moderate on spending
            "user_reply_conv3": "strongly_against"     # Position 2: Strong anti-spending (CHANGE HERE)
        }
    }
}

# Mock significant change (what your detector would return)
mock_significant_change = {
    'position': 2,
    'utterance_id': 'user_reply_conv3',  # The utterance where change was detected
    'p_value': 0.023,
    'p_corrected': 0.041,
    'magnitude': 1.5,
    'left_mean': -0.5,   # Was moderate
    'right_mean': -2.0,  # Became strongly against
    'statistically_significant': True,
    'survives_fdr_correction': True
}

print("Mock data created!")
print("Timeline:", mock_timelines["TestUser"]["taxation and government spending"])
print("Significant change detected at:", mock_significant_change['utterance_id'])

Mock data created!
Timeline: {'user_reply_conv1': 'moderately_against', 'user_reply_conv2': 'neutral', 'user_reply_conv3': 'strongly_against'}
Significant change detected at: user_reply_conv3


In [ ]:
# Test WindowExtractor with mock data
mock_corpus = MockCorpus()

# Create extractor with mock data
extractor = ConversationWindowExtractor(mock_corpus, mock_timelines)

# Test extraction around the significant change
window_data = extractor.get_conversations_around_change(
    user_id="TestUser",
    topic="taxation and government spending",
    change=mock_significant_change,
    window_size=2
)

rooted_units_data = extractor.extract_rooted_path_units_from_window(window_data)


# Preprocessing
interplay_data = extractor.preprocess_rooted_units_for_interplay(rooted_units_data)

# Print results
if window_data:
    extractor.print_window_summary(window_data)

    # Show actual conversation content
    print("\n=== Conversation Content ===")

    print("\nBEFORE change conversations:")
    for i, conv in enumerate(window_data['before_change']):
        print(f"\nConversation {i+1} ({conv['conversation_id']}):")
        print(f"OP: {conv['op_post'].text[:100]}...")
        print(f"User: {conv['user_replies'][0].text[:100]}...")

    print("\nAFTER change conversations:")
    for i, conv in enumerate(window_data['after_change']):
        print(f"\nConversation {i+1} ({conv['conversation_id']}):")
        print(f"OP: {conv['op_post'].text[:100]}...")
        print(f"User: {conv['user_replies'][0].text[:100]}...")

else:
    print("No window data extracted - check for errors")



Found change at position 2 of 3 total utterances
Looking for conversations before change: positions 0 to 1
  Found conversation before: conv_1
  Found conversation before: conv_2
Looking for conversations after change: positions 3 to 2

=== Conversation Window Summary ===
Change occurred at position 2 of 3 utterances
Change utterance ID: user_reply_conv3
Window size: 2

Conversations BEFORE change: 2
  1. Conversation conv_1: 1 user replies
  2. Conversation conv_2: 1 user replies

Conversations AFTER change: 0

=== Conversation Content ===

BEFORE change conversations:

Conversation 1 (conv_1):
OP: I think we should raise taxes on the wealthy to fund infrastructure. The current system isn't workin...
User: I disagree with raising taxes. The wealthy already pay their fair share and higher taxes will hurt e...

Conversation 2 (conv_2):
OP: Government healthcare spending is out of control. We need to cut Medicare and focus on private solut...
User: Medicare is essential for seniors. We s

In [78]:
# Detect persistent changes:
timeline_builder = TimelineBuilder(corpus, min_posts_per_topic=0, min_topics_per_user=0)
timelines = timeline_builder.build_timelines()

# Filter for analysis
filtered_timelines = filter_for_change_detection(timelines, min_posts_per_topic=5, min_topics_per_user=2)

# Convert to list of tupples
topic_timeline_list = list(topic_timeline.items())

# Get a specific user's timeline for a specific topic
user_id = "HardCoreModerate"
topic = "media and political commentary"
topic_timeline = filtered_timelines[user_id][topic]  # This is {utterance_id: stance}

persistence_detector = ChangeDetector()

2025-08-03 13:06:26,778 - temporal_belief.core.timeline_building - INFO - timeline_building:71 - Built timelines for 4781 users
INFO:temporal_belief.core.timeline_building:Built timelines for 4781 users


In [79]:
change_points = persistence_detector.detect_persistent_changes(topic_timeline_list)

Current:moderately_against, Previous: moderately_favor and Next:moderately_against


In [87]:
window_extractor = ConversationWindowExtractor(corpus, timelines=timelines)
change_points = persistence_detector.detect_persistent_changes(topic_timeline_list)
convos = window_extractor.get_conversation_around_change_point(change_points=change_points, corpus=corpus)
for convo in convos:
  print(f"Conversation_id: {convo.id}")

Current:moderately_against, Previous: moderately_favor and Next:moderately_against


UnboundLocalError: cannot access local variable 'conversation_id' where it is not associated with a value

In [52]:
print(change_points)

(141, 'c3dpsmt')


In [49]:
print(topic_timeline_list[0][0])

lnrey


In [ ]:
# Detect changes with significance:
timeline_builder = TimelineBuilder(corpus, min_posts_per_topic=0, min_topics_per_user=0)
timelines = timeline_builder.build_timelines()

# Filter for analysis
filtered_timelines = filter_for_change_detection(timelines, min_posts_per_topic=5, min_topics_per_user=2)

# Get a specific user's timeline for a specific topic
user_id = "HardCoreModerate"
topic = "media and political commentary"
topic_timeline = filtered_timelines[user_id][topic]  # This is {utterance_id: stance}

# Initialize detector and detect changes
detector = ChangeDetector()
significant_changes, p_values, p_corrected = detector.detect_changes_with_significance(topic_timeline)

# Print the results
print(f"Detected {len(significant_changes)} statistically significant stance changes for user {user_id} on topic {topic}:")
for change in significant_changes:
    print(f"  {change['stance_before']} → {change['stance_after']} (magnitude: {change['magnitude']:.3f}, p={change['p_corrected']:.4f})")

Detected 0 statistically significant stance changes for user HardCoreModerate on topic media and political commentary:


In [ ]:
# Test window extractor
user_id = "HardCoreModerate"
topic = "taxation and government spending"
topic_timeline = timelines[user_id][topic]

# Get significant changes
detector = ChangeDetector()
significant_changes, p_values, p_corrected = detector.detect_changes_with_significance(topic_timeline)

if significant_changes:
    # Test the window extractor
    extractor = ConversationWindowExtractor(corpus, timelines)
    window_data = extractor.get_conversations_around_change(
        user_id=user_id,
        topic=topic,
        change=significant_changes[0],
        window_size=2  # 2 conversations before + 2 after
    )

    # Print summary
    extractor.print_window_summary(window_data)
else:
    print("No significant changes found to test with")

No significant changes found to test with


In [ ]:
# Most populated topic for a user
def topic_with_most_contributions(user_id):
    posts_in_topic = {}
    for topic in timelines[user_id].keys():
      posts_in_topic[topic] = len(list(timelines[user_id][topic]))
    # key with the largest value
    topic = max(posts_in_topic, key=posts_in_topic.get)

    return topic, posts_in_topic[topic]

# Yea the number came cause the posts_in_topic was not encapsulated
user_id = 'HardCoreModerate'
topic, number = topic_with_most_contributions(user_id)
print(f"{topic}: {number}")
# print(posts_in_topic)

media and political commentary: 145


In [ ]:
# Total number of users with metadata (unfiltered)
print(len(timelines))

In [ ]:
# NOT WORKING
# user with the most utterances:
# I have to find the max between their topics and then find the overall max
users = {}
for user_id, data in timelines.items():
    topic, number = topic_with_most_contributions(user_id)
    users[user_id] = topic
    users[user_id][topic] = number

for user in users:
    print(user)
# user_id = max(users, key=users.get)
# print(f"{user_id}: {users[user_id]}")

In [ ]:
# Detect simple stance change:
timeline_builder = TimelineBuilder(corpus, min_posts_per_topic=3, min_topics_per_user=1)
timelines = timeline_builder.build_timelines()

# Get a specific user's timeline for a specific topic
user_id = "HardCoreModerate"
topic = "taxation and government spending"
topic_timeline = timelines[user_id][topic]  # This is {utterance_id: stance}

# Initialize detector and detect changes
detector = ChangeDetector()
changes = detector.detect_simple_stance_changes(topic_timeline)

# Print the results
print(f"Detected {len(changes)} stance changes for user {user_id} on topic {topic}:")
for change in changes:
    print(f"  {change['from_stance']} → {change['to_stance']} (magnitude: {change['change_magnitude']})")

Detected 2 stance changes for user HardCoreModerate on topic taxation and government spending:
  moderately_against → neutral (magnitude: 1)
  neutral → moderately_against (magnitude: 1)


In [ ]:
# for user_id in

In [ ]:
# Run detection for all topics for a user - NOT TESTED:
# Get complete user timeline
user_timeline = timelines["pixel8"]  # All topics for this user

# Analyze changes across all topics
detector = ChangeDetector()
all_changes = detector.analyze_user_belief_changes(user_timeline)

# Results
for topic, changes in all_changes.items():
    print(f"Topic: {topic}")
    for change in changes:
        print(f"  Change at position {change['position']}: magnitude {change['magnitude']}")

In [ ]:
# All users that meet the criteria:
print("Available users:")
print(list(timelines.keys())[:20])

In [ ]:
# What topics the users have posted about:
for user_id in list(timelines.keys())[:20]:  # Check first 5 users
    topics = list(timelines[user_id].keys())
    print(f"{user_id}: {topics}")
    break

In [ ]:
# confidence score:
utterances = list(corpus.iter_utterances())
print(utterances[1].meta)

ConvoKitMeta({'score': 29, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'PoliticalDiscussion', 'stickied': False, 'permalink': '/r/PoliticalDiscussion/comments/nz1xu/congrats_rpoliticaldiscussion_you_are_turning/', 'author_flair_text': '', 'detected_stance': 'moderately_against', 'stance_confidence': 0.8540321985880533, 'stance_scores': {'strongly_favor': 0.0016047263949682626, 'moderately_favor': 0.5134096046288809, 'neutral': 0.0072105322033166885, 'moderately_against': 0.8540321985880533, 'strongly_against': 0.3021060957883795}})
